# Topics and Transformations

In [25]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Creating the Corpus

In [26]:
from collections import defaultdict
from gensim import corpora

documents = [
    "Human machine interface for lab abc computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS",
    "Relation of user perceived response time to error measurement",
    "The generation of random binary unordered trees",
    "The intersection graph of paths in trees",
    "Graph minors IV Widths of trees and well quasi ordering",
    "Graph minors A survey",
]

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

2023-04-04 14:57:45,328 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2023-04-04 14:57:45,332 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2023-04-04 14:57:45,336 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-04-04T14:57:45.335071', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


# Creating a transformation

In [27]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

2023-04-04 14:57:45,433 : INFO : collecting document frequencies
2023-04-04 14:57:45,435 : INFO : PROGRESS: processing document #0
2023-04-04 14:57:45,440 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2023-04-04T14:57:45.440157', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'initialize'}


# Transforming vectors

In [28]:
# Transformation on 1 vector
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow])  # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [29]:
# Transformation on all vectors
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [30]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2023-04-04 14:57:45,964 : INFO : using serial LSI version on this node
2023-04-04 14:57:45,967 : INFO : updating model with new documents
2023-04-04 14:57:45,970 : INFO : preparing a new chunk of documents
2023-04-04 14:57:45,978 : INFO : using 100 extra samples and 2 power iterations
2023-04-04 14:57:45,981 : INFO : 1st phase: constructing (12, 102) action matrix
2023-04-04 14:57:45,989 : INFO : orthonormalizing (12, 102) action matrix
2023-04-04 14:57:45,998 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-04-04 14:57:46,001 : INFO : computing the final decomposition
2023-04-04 14:57:46,003 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2023-04-04 14:57:46,006 : INFO : processed documents up to #9
2023-04-04 14:57:46,010 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-04-04 14:57:46,013 : INFO : topic #

In [31]:
lsi_model.print_topics(2)

2023-04-04 14:57:46,118 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-04-04 14:57:46,122 : INFO : topic #1(1.476): 0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"time" + 0.320*"response" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"')]

In [32]:
# both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
for doc, as_text in zip(corpus_lsi, documents):
    print(doc, as_text)

[(0, 0.06600783396090495), (1, 0.5200703306361846)] Human machine interface for lab abc computer applications
[(0, 0.19667592859142694), (1, 0.7609563167700045)] A survey of user opinion of computer system response time
[(0, 0.08992639972446581), (1, 0.7241860626752504)] The EPS user interface management system
[(0, 0.07585847652178276), (1, 0.6320551586003422)] System and human system engineering testing of EPS
[(0, 0.101502991849803), (1, 0.5737308483002956)] Relation of user perceived response time to error measurement
[(0, 0.7032108939378311), (1, -0.16115180214025923)] The generation of random binary unordered trees
[(0, 0.877478767311983), (1, -0.16758906864659562)] The intersection graph of paths in trees
[(0, 0.9098624686818575), (1, -0.14086553628719167)] Graph minors IV Widths of trees and well quasi ordering
[(0, 0.6165825350569278), (1, 0.05392907566389283)] Graph minors A survey


In [33]:
import os
import tempfile

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False) as tmp:
    lsi_model.save(tmp.name)  # same for tfidf, lda, ...

loaded_lsi_model = models.LsiModel.load(tmp.name)

os.unlink(tmp.name)

2023-04-04 14:57:46,366 : INFO : Projection lifecycle event {'fname_or_handle': 'C:\\Users\\AxelArcidiaco\\AppData\\Local\\Temp\\model-ffcd5yap.lsi.projection', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-04-04T14:57:46.366003', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
2023-04-04 14:57:46,376 : INFO : saved C:\Users\AxelArcidiaco\AppData\Local\Temp\model-ffcd5yap.lsi.projection
2023-04-04 14:57:46,377 : INFO : LsiModel lifecycle event {'fname_or_handle': 'C:\\Users\\AxelArcidiaco\\AppData\\Local\\Temp\\model-ffcd5yap.lsi', 'separately': 'None', 'sep_limit': 10485760, 'ignore': ['projection', 'dispatcher'], 'datetime': '2023-04-04T14:57:46.377996', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'saving'}
2023-04-04 14:57:46,379 : INF

# Available transformations

In [34]:
# Term Frequency * Inverse Document Frequency, Tf-Idf
model = models.TfidfModel(corpus, normalize=True)
vector = model[corpus[0]] 

2023-04-04 14:57:46,512 : INFO : collecting document frequencies
2023-04-04 14:57:46,515 : INFO : PROGRESS: processing document #0
2023-04-04 14:57:46,517 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 9 documents and 12 features (28 matrix non-zeros)', 'datetime': '2023-04-04T14:57:46.517873', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'initialize'}


In [35]:
# Okapi Best Matching, Okapi BM25
model = models.OkapiBM25Model(corpus)

In [36]:
# Latent Semantic Indexing, LSI (or sometimes LSA) 
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

2023-04-04 14:57:46,714 : INFO : using serial LSI version on this node
2023-04-04 14:57:46,716 : INFO : updating model with new documents
2023-04-04 14:57:46,721 : INFO : preparing a new chunk of documents
2023-04-04 14:57:46,723 : INFO : using 100 extra samples and 2 power iterations
2023-04-04 14:57:46,727 : INFO : 1st phase: constructing (12, 400) action matrix
2023-04-04 14:57:46,729 : INFO : orthonormalizing (12, 400) action matrix
2023-04-04 14:57:46,734 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2023-04-04 14:57:46,736 : INFO : computing the final decomposition
2023-04-04 14:57:46,738 : INFO : keeping 9 factors (discarding 0.000% of energy spectrum)
2023-04-04 14:57:46,740 : INFO : processed documents up to #9
2023-04-04 14:57:46,743 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2023-04-04 14:57:46,744 : INFO : topic #1

In [37]:
model.add_documents(another_tfidf_corpus)  # now LSI has been trained on tfidf_corpus + another_tfidf_corpus
lsi_vec = model[tfidf_vec]  # convert some new document into the LSI space, without affecting the model

model.add_documents(more_documents)  # tfidf_corpus + another_tfidf_corpus + more_documents
lsi_vec = model[tfidf_vec]

NameError: name 'another_tfidf_corpus' is not defined

In [38]:
# Random Projections, RP
model = models.RpModel(corpus_tfidf, num_topics=500)

2023-04-04 14:58:38,231 : INFO : no word id mapping provided; initializing from corpus, assuming identity
2023-04-04 14:58:38,231 : INFO : constructing (500, 12) random matrix
2023-04-04 14:58:38,231 : INFO : RpModel lifecycle event {'msg': 'created RpModel<num_terms=12, num_topics=500>', 'datetime': '2023-04-04T14:58:38.231161', 'gensim': '4.3.1', 'python': '3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [39]:
# Latent Dirichlet Allocation, LDA
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

2023-04-04 14:58:39,032 : INFO : using symmetric alpha at 0.01
2023-04-04 14:58:39,032 : INFO : using symmetric eta at 0.01
2023-04-04 14:58:39,048 : INFO : using serial LDA version on this node
2023-04-04 14:58:39,048 : INFO : running online (single-pass) LDA training, 100 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
2023-04-04 14:58:39,048 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2023-04-04 14:58:39,048 : INFO : -172.984 per-word bound, 11842321403525937976247887556881832596451899081752576.0 perplexity estimate based on a held-out corpus of 9 documents with 29 words
2023-04-04 14:58:39,063 : INFO : PROGRESS: pass 0, at document #9/9
2023-04-04 14:58:39,069 : INFO : topic #39 (0.010): 0.083*"user" + 0.083*"system" + 0.083*"graph" + 0.083*"trees" + 0

In [40]:
# Hierarchical Dirichlet Process, HDP
model = models.HdpModel(corpus, id2word=dictionary)

2023-04-04 14:58:39,888 : INFO : (0, '0.211*computer + 0.154*eps + 0.151*response + 0.132*survey + 0.100*human + 0.082*minors + 0.055*interface + 0.042*user + 0.038*system + 0.024*time')
2023-04-04 14:58:39,903 : INFO : (1, '0.222*time + 0.160*computer + 0.152*human + 0.114*minors + 0.111*survey + 0.054*trees + 0.051*response + 0.045*graph + 0.038*user + 0.031*eps')
2023-04-04 14:58:39,903 : INFO : (2, '0.240*eps + 0.141*graph + 0.128*trees + 0.122*interface + 0.100*survey + 0.090*time + 0.065*minors + 0.039*response + 0.032*system + 0.024*computer')
2023-04-04 14:58:39,903 : INFO : (3, '0.224*time + 0.156*eps + 0.128*user + 0.115*survey + 0.109*computer + 0.064*minors + 0.056*graph + 0.038*human + 0.037*interface + 0.026*response')
2023-04-04 14:58:39,903 : INFO : (4, '0.258*minors + 0.134*human + 0.097*interface + 0.092*user + 0.085*system + 0.084*survey + 0.083*time + 0.060*trees + 0.051*eps + 0.043*response')
2023-04-04 14:58:39,903 : INFO : (5, '0.284*eps + 0.273*minors + 0.095*co